In [1]:
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
import os
import logging
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [43]:
TOYOTA_SYSTEM_PROMPT = """
You are a friendly and knowledgeable Toyota AI Sales Assistant for Toyota Automobiles North America. Your role is to help customers explore Toyota vehicles, find inventory near their location, and schedule test drives.

**Your Personality:**
- Enthusiastic about Toyota vehicles and their features
- Professional yet conversational and approachable
- Knowledgeable about Toyota's complete lineup
- Proactive in suggesting next steps
- Honest and transparent about vehicle information

**Your Capabilities:**
- Search for Toyota vehicle information and specifications
- Find available inventory near customer ZIP codes
- Help schedule test drives at local dealerships
- Provide detailed vehicle comparisons and recommendations
- Remember customer preferences throughout the conversation

**Guidelines:**
1. **Toyota Focus Only:** Only discuss Toyota vehicles, features, and services
2. **Gather Information:** Ask for customer preferences (budget, needs, location) early in conversation
3. **Use Tools Wisely:** 
   - Use serper_search_tool for general information and FAQs
   - do not use tool if inputs are not available for the tool
   - Use search_toyota_info for specifications, reviews, and features
   - Use search_inventory to find available vehicles near customers
   - Use get_vehicle_details for specific vehicle information
   - Use save_test_drive when customer wants to book an appointment
4. **Be Proactive:** Always suggest relevant next steps like viewing inventory or scheduling test drives
5. **Remember Context:** Keep track of customer preferences, location, and interests
6. **Accurate Information:** Only provide verified information from your tools
7. **Professional Communication:** Use proper formatting and clear explanations

**Conversation Flow:**
1. Greet warmly and ask how you can help
2. Gather customer needs (type of vehicle, budget, location, priorities)
3. Show relevant vehicles from inventory when possible
4. Provide detailed information using available tools
5. Proactively suggest test drives for interested customers
6. Help with scheduling and next steps

**Important Notes:**
- Always verify availability before suggesting vehicles
- Include dealership contact information when showing inventory
- Mention key Toyota features like Toyota Safety Sense, hybrid options, reliability
- Be enthusiastic about Toyota's innovation and quality
- If you don't have specific information, use the search tool or direct to official Toyota resources

Remember: Your goal is to help customers find their perfect Toyota vehicle and create a smooth path to ownership!
"""


In [ ]:
from langchain_core.tools import Tool
from langchain_community.utilities import GoogleSerperAPIWrapper
import json

SERPER_API_KEY = os.getenv("SERPER_API_KEY")
from database_setup import (
    query_db,
    insert_data,
    update_data,
    get_inventory_by_zipcode,
)

load_dotenv()
SERPER_API_KEY = os.getenv("SERPER_API_KEY")


def serper_search_tool(json_input: str) -> str:
    try:
        payload = json.loads(json_input)
        q = payload.get("q", "").strip()
        if not q:
            return json.dumps({"error": "missing_query"})
        if not SERPER_API_KEY or GoogleSerperAPIWrapper is None:
            return json.dumps({
                "ok": True,
                "text": f"No SERPER configured. For '{q}', check toyota.com or official resources.",
            })
        search = GoogleSerperAPIWrapper()
        out = search.run(q)
        return json.dumps({"ok": True, "text": out})
    except Exception as e:
        return json.dumps({"error": "serper_failed", "detail": str(e)})


def vehicle_search_tool(json_input: str) -> str:
    try:
        q = json.loads(json_input)
    except Exception:
        return json.dumps({"error": "invalid_json"})
    print("vehicle_search_tool input:", q)
    model = q.get("model")
    trim = q.get("trim")
    color = q.get("color")
    zipcode = q.get("zipcode")
    features = q.get("features", []) or []

    try:
        inv = (
            get_inventory_by_zipcode(zipcode, model)
            if (zipcode or model)
            else get_inventory_by_zipcode(None, None)
        )
        results = []
        for row in inv:
            (
                vid,
                v_model,
                v_trim,
                v_color,
                v_rate,
                v_features,
                d_name,
                d_city,
                d_zip,
                d_addr,
                d_phone,
                inv_id,
                vin,
            ) = row

            if trim and str(v_trim).lower() != str(trim).lower():
                continue
            if color and str(v_color).lower() != str(color).lower():
                continue

            try:
                feat = json.loads(v_features) if v_features else {}
            except Exception:
                feat = {}

            if features:
                joined = json.dumps(feat).lower()
                if not any(str(f).lower() in joined for f in features):
                    continue

            results.append(
                {
                    "vehicle": {
                        "vehicle_id": vid,
                        "make": "Toyota",
                        "model": v_model,
                        "trim": v_trim,
                        "color": v_color,
                        "price": v_rate,
                        "features": feat,
                        "vin": vin,
                    },
                    "dealership": {
                        "name": d_name,
                        "address": d_addr,
                        "city": d_city,
                        "zipcode": d_zip,
                        "phone": d_phone,
                    },
                    "inventory_id": inv_id,
                }
            )
        return json.dumps({"ok": True, "results": results}, default=str)
    except Exception as e:
        return json.dumps({"error": "inventory_query_failed", "detail": str(e)})


def save_booking_tool(json_input: str) -> str:
    try:
        payload = json.loads(json_input)
    except Exception:
        return json.dumps({"error": "invalid_json"})

    cust = payload.get("customer", {})
    vehicle = payload.get("vehicle", {})
    dealership_id = payload.get("dealership_id")
    salesperson_id = payload.get("salesperson_id")
    inventory_id = payload.get("inventory_id")
    date_s = payload.get("date")
    time_s = payload.get("time")
    special = payload.get("special_request", "")

    # Resolve dealership_id and/or vehicle_id from inventory if provided
    try:
        if (not dealership_id or not vehicle.get("vehicle_id")) and inventory_id:
            inv_rows = query_db(
                "SELECT dealership_id, vehicle_id FROM Inventory WHERE id = ?",
                (inventory_id,),
            )
            if inv_rows:
                inv_dealer_id, inv_vehicle_id = inv_rows[0]
                dealership_id = dealership_id or inv_dealer_id
                if not vehicle.get("vehicle_id"):
                    vehicle["vehicle_id"] = inv_vehicle_id
    except Exception:
        # Continue; required check below will handle any missing pieces
        pass

    required = [cust.get("email"), vehicle.get("vehicle_id"), dealership_id, date_s, time_s]
    if not all(required):
        return json.dumps({"error": "missing_required_fields"})

    try:
        existing = query_db("SELECT customer_id FROM Customer WHERE email = ?", (cust.get("email"),))
        if existing:
            customer_id = existing[0][0]
            update_data(
                "UPDATE Customer SET customer_name=?, phone=?, city=?, zipcode=? WHERE customer_id=?",
                (
                    cust.get("name"),
                    cust.get("phone"),
                    cust.get("city"),
                    cust.get("zipcode"),
                    customer_id,
                ),
            )
        else:
            customer_id = insert_data(
                "INSERT INTO Customer (customer_name, email, phone, zipcode, city) VALUES (?, ?, ?, ?, ?)",
                (
                    cust.get("name"),
                    cust.get("email"),
                    cust.get("phone"),
                    cust.get("zipcode"),
                    cust.get("city"),
                ),
            )

        if not customer_id:
            return json.dumps({"error": "customer_save_failed"})

        created = datetime.utcnow().isoformat()
        testdrive_id = insert_data(
            (
                "INSERT INTO TestDrive (customer_id, dealership_id, salesperson_id, vehicle_id, date, time, special_request, status, created_at) "
                "VALUES (?, ?, ?, ?, ?, ?, ?, 'scheduled', ?)"
            ),
            (
                customer_id,
                dealership_id,
                salesperson_id,
                vehicle.get("vehicle_id"),
                date_s,
                time_s,
                special,
                created,
            ),
        )

        if not testdrive_id:
            return json.dumps({"error": "booking_save_failed"})

        # If we know the inventory row, reserve it to avoid double booking
        if inventory_id:
            try:
                update_data(
                    "UPDATE Inventory SET available_status = 'reserved' WHERE id = ?",
                    (inventory_id,),
                )
            except Exception:
                # Non-fatal; booking still succeeded
                pass

        return json.dumps({"ok": True, "testdrive_id": testdrive_id, "inventory_id": inventory_id})
    except Exception as e:
        return json.dumps({"error": "save_failed", "detail": str(e)})


def send_email_tool(json_input: str) -> str:
    try:
        payload = json.loads(json_input)
    except Exception:
        return json.dumps({"error": "invalid_json"})

    to = payload.get("to")
    subject = payload.get("subject", "Toyota Test Drive")
    html = payload.get("html", "")

    try:
        from notifications import send_email as send_email_fn
    except Exception as e:
        return json.dumps({"error": "email_helper_missing", "detail": str(e)})

    ok = send_email_fn(to, subject, html, is_html=True)
    return json.dumps({"ok": bool(ok), "to": to})


def get_vehicle_details_tool(json_input: str) -> str:
    try:
        payload = json.loads(json_input)
        vid = payload.get("vehicle_id")
        if not vid:
            return json.dumps({"error": "missing_vehicle_id"})
    except Exception:
        return json.dumps({"error": "invalid_json"})

    try:
        query = (
            "SELECT v.id, v.make, v.model, v.trim, v.color, v.rate, v.features, "
            "d.dealership_name, d.address, d.city, d.zipcode, d.phone, i.vin "
            "FROM Vehicle v JOIN Inventory i ON i.vehicle_id = v.id "
            "JOIN Dealership d ON i.dealership_id = d.dealership_id "
            "WHERE v.id = ? AND i.available_status = 'available' LIMIT 1"
        )
        rows = query_db(query, (vid,))
        if not rows:
            return json.dumps({"error": "not_found"})
        (
            v_id,
            make,
            model_name,
            trim,
            color,
            rate,
            features_json,
            dealership_name,
            address,
            city,
            zipcode,
            phone,
            vin,
        ) = rows[0]

        try:
            features = json.loads(features_json) if features_json else {}
        except Exception:
            features = {}

        result = {
            "vehicle": {
                "id": v_id,
                "make": make,
                "model": model_name,
                "trim": trim,
                "color": color,
                "price": rate,
                "features": features,
                "vin": vin,
            },
            "dealership": {
                "name": dealership_name,
                "address": address,
                "city": city,
                "zipcode": zipcode,
                "phone": phone,
            },
        }
        return json.dumps({"ok": True, "result": result}, default=str)
    except Exception as e:
        return json.dumps({"error": "db_failed", "detail": str(e)})


# LangChain Tool wrappers
serper_tool = Tool(
    name="search_toyota_info",
    func=lambda q: serper_search_tool(json.dumps({"q": q})),
    description=(
        "Search Toyota info/specs/reviews via Serper (falls back gracefully without API key). "
        "Input: plain query string."
    ),
)



booking_tool = Tool(
    name="save_test_drive",
    func=save_booking_tool,
    description=(
        "Save test drive booking. Input JSON: {customer:{name,email,phone,city,zipcode}, "
        "vehicle:{vehicle_id}, dealership_id OR inventory_id, salesperson_id, date, time, special_request}."
    ),
)

vehicle_details_tool = Tool(
    name="get_vehicle_details",
    func=lambda s: get_vehicle_details_tool(s if isinstance(s, str) else json.dumps({"vehicle_id": s})),
    description="Get vehicle details. Input JSON: {vehicle_id} or just vehicle_id as string.",
)

send_email = Tool(
    name="send_email",
    func=send_email_tool,
    description="Send HTML email via configured SMTP (delegates to notifications.py). Input JSON: {to, subject, html}.",
)
def inventory_search_wrapper(input_str: str) -> str:
    """
    Wrapper for vehicle_search_tool.
    Accepts either:
      - A ZIP code string like "90012"
      - Or a JSON string with filters like {"zipcode": "90012", "model": "Camry"}
    """
    try:
        # Try to interpret input as a ZIP code
        if input_str.isdigit():
            payload = json.dumps({"zipcode": input_str})
        else:
            # If it's JSON-like, load to validate
            try:
                json.loads(input_str)
                payload = input_str
            except json.JSONDecodeError:
                # Fallback: treat plain text query as search term
                payload = json.dumps({"zipcode": None, "model": input_str})
        # Call your main function
        return vehicle_search_tool(payload)
    except Exception as e:
        return json.dumps({"error": f"inventory_search_failed: {str(e)}"})
# Export list for agent creation
inventory_tool = Tool.from_function(
    func=inventory_search_wrapper,
    name="search_inventory",
    description="Search Toyota vehicle inventory by ZIP or model. Input can be a ZIP code or JSON string with keys {zipcode, model, trim, color, features}."
)
tools = [serper_tool, inventory_tool]

In [54]:
 llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0.7,
        openai_api_key=os.getenv("OPENAI_API_KEY"),
        timeout=60,
        max_retries=2,
    )
memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True,
        output_key="output",
    )

    # Initialize agent with ReAct framework

agent = initialize_agent(
    tools=tools,
        llm=llm,
        agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
        memory=memory,
        verbose=True,
        agent_kwargs={"system_message": TOYOTA_SYSTEM_PROMPT},
        handle_parsing_errors=True,
        max_iterations=5,
    )


In [55]:

user_input = "tell me about Camry sedan car models and tell me XSE trim with features"
result = agent.run({"input": user_input})




> Entering new AgentExecutor chain...


ValueError: Missing some input keys: {'zipcode, model, trim, color, features'}